In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 9)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-09
yesterday: 2023-06-08


### Tables in the process

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [20]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,ORI,125,2023-05-17,"3,000",10.20,1030,1254,2023-06-09,11.20,69.55,4.87,"33,525.58",23,"2,857.81",9.32%,147.88%,-30,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [21]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'ORI'

In [22]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1254



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,ORI,1254,125,2023-05-17,"3,000",10.20,63.34,4.43,Sold,"30,667.77",DTD,None


In [23]:
transaction, name, qty, sell_price, buy_price

('S', 'ORI', -3000, 11.2, 10.2)

### In case of buy, retreive buy id from latest buy record (id)

In [32]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,AH,1261,209,2023-06-08,"1,200",37.00,91.91,6.43,Active,"44,498.34",DTD,None


In [33]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'AH', 1200, 37.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [24]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'ORI'



,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"48,000",10.95,1,2,A2


In [25]:
new_cost = 11
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -3000, price = 11
WHERE name = 'ORI'



In [26]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [27]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'ORI'



,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"45,000",11.00,1,2,A2


### If shares become zero, must delete from buy table

In [26]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'KTC' AND volbuy = 0


In [27]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [28]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ORI'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.5700,0.0000,0.1500,0.0000,0.7200,"48,000",2023-05-08,2023-05-25,3,0


In [29]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -3000
WHERE name = 'ORI'



In [30]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [31]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ORI'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.5700,0.0000,0.1500,0.0000,0.7200,"45,000",2023-05-08,2023-05-25,3,0


### If shares become zero and want to delete from DIVIDEND table

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s'
"""
sqlDel = sqlDel % (name)
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [32]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'ORI'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,716,ORI,12.70,8.05,B,10.10,11.10,116.98,1.32,10.95,"3,000",-7,8,"48,000",0,0,RD05pct,SET100


In [33]:
buy_target  = 10 # must calculate manually beforehand
sell_target = 11.6
cost = 11
status = 'O'
reason = "52WL"
sell_qty = 3000

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -3000,
qty = 3000, buy_target = 10, sell_target = 11.6, cost = 11, reason = '52WL', status = 'O'
WHERE name = 'ORI'



In [34]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [35]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'ORI'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,716,ORI,12.70,8.05,O,10.00,11.60,116.98,1.32,11.00,"3,000",-7,8,"45,000",0,0,52WL,SET100


### End of Update stocks table in SQLite port_lite database from sale transaction

### If shares become zero and no longer wants to keep STOCK record

In [ ]:
sqlDel = """
DELETE FROM stocks
WHERE name = '%s' AND available_qty <= 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = conlite.execute(sqlDel)
rp.rowcount

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [34]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'AH'



,name,date,shares,price,active,dividend,period,grade


### In case there is the same stock name in port

In [10]:
new_cost = 37 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 600, price = 53.5
WHERE name = 'JMT'



In [11]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [12]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMT'



,name,date,shares,price,active,dividend,period,grade
0,JMT,2023-01-26,"4,800",53.50,1,0.9900,3,A4


### In case of totally new stock

In [35]:
rcd = ['AH', '2023-06-08', 1200, 37, 0, 0, 1, 0, 4,'A3']
rcd

['AH', '2023-06-08', 1200, 37, 0, 0, 1, 0, 4, 'A3']

In [36]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [37]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [38]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'AH'



,name,date,shares,price,active,dividend,period,grade
0,AH,2023-06-08,"1,200",37.00,1,0.0000,4,A3


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [13]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMT,0.5900,0.0000,0.4000,0.0000,0.9900,"4,200",2023-04-11,2023-05-02,,0


In [14]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 600
WHERE name = 'JMT'



In [15]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [16]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMT,0.5900,0.0000,0.4000,0.0000,0.9900,"4,800",2023-04-11,2023-05-02,,0


### In case of totally new stock

In [ ]:
rcd = ['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]
rcd

In [ ]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [39]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks


SELECT * 
FROM stocks 
WHERE name = 'AH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9773,AH,0,0,O,19.4,0,0,0,0,1200,-26,33,0,0,0,52WL,SET


In [40]:
new_cost = 37
buy_target = 37
sell_target = 39
status = 'I'
reason = '5pct'
new_buy_qty = 1200
new_sell_qty = 1200
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(37, 37, 39, 'I', '5pct', 1200, 1200)

In [41]:
#qty = 12000
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 1200, 
cost = 37, buy_target = 37, sell_target = 39, status = 'I', 
reason = '5pct', qty = 1200
WHERE name = 'AH'



In [42]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [43]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'AH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9773,AH,0.00,0.00,I,37.00,39.00,0.00,0.00,37.00,"1,200",-26,33,"1,200",0,0,5pct,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

In [36]:
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2023-06-08' ORDER BY period, buy.name


In [37]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,JASIF,2018-05-17,"130,000",10.00,7.00,"1,300,000.00","910,000.00","-390,000.00",-30.00
1,1,C4,MCS,2016-09-20,"78,000",15.10,7.00,"1,177,800.00","546,000.00","-631,800.00",-53.64
2,1,C1,PTTGC,2021-03-17,"6,000",64.75,38.25,"388,500.00","229,500.00","-159,000.00",-40.93
3,1,C2,TMT,2021-08-16,"36,000",10.20,7.45,"367,200.00","268,200.00","-99,000.00",-26.96
4,2,A4,ASK,2023-02-16,"4,500",31.00,28.25,"139,500.00","127,125.00","-12,375.00",-8.87
5,2,C4,ASP,2022-03-07,"30,000",3.80,2.86,"114,000.00","85,800.00","-28,200.00",-24.74
6,2,A3,BANPU,2023-01-06,"30,000",11.50,8.65,"345,000.00","259,500.00","-85,500.00",-24.78
7,2,A2,CPNREIT,2022-08-16,"60,000",17.55,12.80,"1,053,000.00","768,000.00","-285,000.00",-27.07
8,2,C1,DIF,2020-08-01,"30,000",14.70,12.40,"441,000.00","372,000.00","-69,000.00",-15.65
9,2,B1,GVREIT,2022-08-24,"30,000",8.90,9.00,"267,000.00","270,000.00","3,000.00",1.12


In [38]:
file_name = 'portfolio.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

output.to_csv(output_file, header=True, index=False)
output.to_csv(data_file, header=True, index=False)
output.to_csv(one_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)